<a href="https://colab.research.google.com/github/LJW0719/CCTV_CNN/blob/main/2_Yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yolo

---

In [ ]:
import cv2
import numpy as np
import time # to calculate frame
from google.colab.patches import cv2_imshow

## Upload Video FIle

In [ ]:
video = cv2.VideoCapture('2021-08-06_10-00-00_fri_sunny_out_ja-ma_C0064.mp4')

## Load Yolo

In [ ]:
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

## Detect and Show Object

### Make 'detectAndShow' Function

In [ ]:
def detectAndShow(frame):
    start_time = time.time()
    img = cv2.resize(frame, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape


    # Set Window Size
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)


    # Detect Object
    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Boxing Detected Object 
            if confidence > 0.5:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
               
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Remove Noise
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

    # Show Results
    font = cv2.FONT_HERSHEY_DUPLEX
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 1, color, 1)
    end_time = time.time()
    process_time = end_time - start_time
    print("=== A frame took {:.3f} seconds".format(process_time))
    cv2_imshow(img)

### Apply Video

In [ ]:
width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)

In [ ]:
while video.isOpened():
  ret, frame = video.read()

  if ret:
    detectAndShow(frame)
    key = cv2.waitKey(10)

    if key == ord('q'):
      break
  else:
    break

video.release()
cv2.destroyAllWindows()

## Detect and Show Only People

### Make 'detectPeople' Function

In [ ]:
def detectPeople(frame):
    start_time = time.time()
    img = cv2.resize(frame, None, fx=0.4, fy=0.4)
    height, width, channels = img.shape

    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)

    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if class_id == 0 and confidence > 0.5: # Change only this part, please input class id
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
               
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    font = cv2.FONT_HERSHEY_DUPLEX
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(classes[class_ids[i]])
            color = colors[i]
            cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(img, label, (x, y + 30), font, 1, color, 1)
    end_time = time.time()
    process_time = end_time - start_time
    print("=== A frame took {:.3f} seconds".format(process_time))
    cv2_imshow(img)

### Apply Video

In [ ]:
width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)

while video.isOpened():
  ret, frame = video.read()

  if ret:
    detectPeople(frame)
    key = cv2.waitKey(10)

    if key == ord('q'):
      break
  else:
    break

video.release()
cv2.destroyAllWindows()